# Streaming Tokens

In this example we will stream tokens from the language model powering an agent.
We will use a ReAct agent as an example. The tl;dr is to use
[streamEvents](https://js.langchain.com/v0.2/docs/how_to/chat_streaming/#stream-events)
([API Ref](https://v01.api.js.langchain.com/classes/langchain_core_runnables.Runnable.html#streamEvents))
and make sure you `stream()` the model within your node.

This how-to guide closely follows the others in this directory, showing how to incorporate the functionality to a prototypical agent in LangGraph.

This works for
[StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html)
and all its subclasses, such as
[MessageGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MessageGraph.html).

<div class="admonition tip">
    <p class="admonition-title">Important</p>
    <p>
        Token streaming supported by many, but not all chat models. Check to see if your LLM integration supports token streaming </code> (<a href="https://js.langchain.com/v0.2/docs/integrations/chat/"> here (doc)</a>). Note that some integrations may support general token streaming but still not support it when performing tool calling.
    </p>
</div>

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        In this how-to, we will create our agent from scratch to be transparent (but verbose). You can accomplish similar functionality using the <code>createReactAgent(model, tools=tool)</code> (<a href="https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html">API doc</a>) constructor. This may be more appropriate if you are used to LangChain’s <a href="https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentexecutor">AgentExecutor</a> class.
    </p>
</div>

## Setup

This guide will use OpenAI's GPT-4o model. We will optionally set our API key
for [LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

---

In [14]:
// Deno.env.set("OPENAI_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "ls__...");
Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Time Travel: LangGraphJS");


## Define the state

The state is the interface for all of the nodes in our graph.


In [15]:
import { BaseMessage } from "@langchain/core/messages";

interface IState {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => BaseMessage[];
    default: () => BaseMessage[];
  };
  next: string;
}

// This defines the agent state
const State: IState = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
};


## Set up the tools

We will first define the tools we want to use. For this simple example, we will
use create a placeholder search engine. However, it is really easy to create
your own tools - see documentation
[here](https://js.langchain.com/v0.2/docs/how_to/custom_tools) on how to do
that.


In [16]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({ query }: { query: string }) => {
    // This is a placeholder for the actual implementation
    return ["Cold, with a low of 3℃"];
  },
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];


[ "Cold, with a low of 3℃" ]

We can now wrap these tools in a simple
[ToolNode](https://langchain-ai.github.io/langgraphjs/reference/classes/prebuilt.ToolNode.html).
This object will actually run the tools (functions) whenever they are invoked by
our LLM.


In [17]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);


## Set up the model

Now we will load the
[chat model](https://js.langchain.com/v0.2/docs/concepts/#chat-models).

1. It should work with messages. We will represent all agent state in the form
   of messages, so it needs to be able to work well with them.
2. It should work with
   [tool calling](https://js.langchain.com/v0.2/docs/how_to/tool_calling/#passing-tools-to-llms),
   meaning it can return function arguments in its response.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        These model requirements are not general requirements for using LangGraph - they are just requirements for this one example.
    </p>
</div>


In [18]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({ model: "gpt-4o" });


After we've done this, we should make sure the model knows that it has these
tools available to call. We can do this by calling
[bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools).


In [19]:
const boundModel = model.bindTools(tools);
boundModel.kwargs;


{
  tools: [
    {
      type: "function",
      function: {
        name: "search",
        description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
        parameters: {
          type: "object",
          properties: [Object],
          required: [Array],
          additionalProperties: false,
          "$schema": "http://json-schema.org/draft-07/schema#"
        }
      }
    }
  ]
}

## Define the graph

We can now put it all together. Time travel requires a checkpointer to save the
state - otherwise you wouldn't have anything go `get` or `update`. We will use
the
[MemorySaver](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MemorySaver.html),
which "saves" checkpoints in-memory.


In [20]:
import { END, START, StateGraph } from "@langchain/langgraph";
import { AIMessageChunk, BaseMessage } from "@langchain/core/messages";
import { RunnableConfig } from "@langchain/core/runnables";

const routeMessage = (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1];
  // If no tools are called, we can finish (respond to the user)
  if (!lastMessage?.tool_calls?.length) {
    return END;
  }
  // Otherwise if there is, we continue and call the tools
  return "tools";
};

const callModel = async (
  state: { messages: Array<BaseMessage> },
  config: RunnableConfig,
) => {
  const { messages } = state;
  const streamOut = await boundModel.stream(messages, config);
  let finalMessage: AIMessageChunk | null = null;
  for await (const chunk of streamOut) {
    if (finalMessage === null) {
      finalMessage = chunk;
    } else {
      finalMessage = finalMessage.concat(chunk);
    }
  }
  return { messages: [finalMessage] };
};

const workflow = new StateGraph({
  channels: State,
});

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
workflow.addNode("tools", toolNode);

// Set the entrypoint as `agent`
workflow.addEdge(START, "agent");
workflow.addConditionalEdges("agent", routeMessage);
workflow.addEdge("tools", "agent");

const graph = workflow.compile();

StateGraph {
  nodes: {
    agent: RunnableLambda {
      lc_serializable: false,
      lc_kwargs: { func: [AsyncFunction: callModel] },
      lc_runnable: true,
      name: undefined,
      lc_namespace: [ "langchain_core", "runnables" ],
      func: [AsyncFunction: callModel]
    },
    tools: ToolNode {
      lc_serializable: false,
      lc_kwargs: {},
      lc_runnable: true,
      name: "tools",
      lc_namespace: [ "langgraph" ],
      func: [Function: func],
      tags: undefined,
      config: { tags: [] },
      trace: true,
      recurse: true,
      tools: [
        DynamicStructuredTool {
          lc_serializable: false,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: "search",
          verbose: false,
          callbacks: undefined,
          tags: [],
          metadata: {},
          returnDirect: false,
          description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
          fu

## Call streamEvents

We can now interact with the agent. Between interactions you can get and update
state.


In [27]:
import { ChatGenerationChunk } from "@langchain/core/outputs";
import { AIMessageChunk } from "@langchain/core/messages";
let config = { configurable: { thread_id: "conversation-num-1" } };
let inputs = { messages: [["user", "Hi I'm Jo."]] };
for await (
  const event of await graph.streamEvents(inputs, {
    ...config,
    streamMode: "values",
    version: "v1",
  })
) {
  if (event.event === "on_llm_stream") {
    let chunk: ChatGenerationChunk = event.data?.chunk;
    let msg = chunk.message as AIMessageChunk;
    if (msg.tool_call_chunks && msg.tool_call_chunks.length > 0) {
      console.log(msg.tool_call_chunks);
    } else {
      console.log(msg.content);
    }
  }
}


Hello
 Jo
!
 How
 can
 I
 assist
 you
 today
?



## How to stream tool calls

Many providers support token-level streaming of tool invocations. To get the
partially populated results, you can access the message chunks'
`tool_call_chunks` property.

In [29]:
for await (
  const event of await graph.streamEvents(
    { messages: [["user", "What's the weather like today?"]] },
    {
      ...config,
      streamMode: "values",
      version: "v1",
    },
  )
) {
  if (event.event === "on_llm_stream") {
    let chunk: ChatGenerationChunk = event.data?.chunk;
    let msg = chunk.message as AIMessageChunk;
    if (msg.tool_call_chunks && msg.tool_call_chunks.length > 0) {
      console.log(msg.tool_call_chunks);
    } else {
      console.log(msg.content);
    }
  }
}

[
  {
    name: "search",
    args: "",
    id: "call_P0UsjSBXstqMbMZpSTWEzxqL",
    index: 0
  }
]
[ { name: undefined, args: '{"', id: undefined, index: 0 } ]
[ { name: undefined, args: "query", id: undefined, index: 0 } ]
[ { name: undefined, args: '":"', id: undefined, index: 0 } ]
[ { name: undefined, args: "current", id: undefined, index: 0 } ]
[ { name: undefined, args: " weather", id: undefined, index: 0 } ]
[ { name: undefined, args: '"}', id: undefined, index: 0 } ]


The
 weather
 today
 is
 cold
,
 with
 a
 low
 temperature
 of
 
3
℃
.
 Stay
 warm
!

